In [ ]:
import os
from owslib.wms import WebMapService
from PIL import Image
import requests
from io import BytesIO
import json

def save_metadata(filename, bbox, size, crs, resolution):
    metadata = {
        "bbox": bbox,
        "size": size,
        "crs": crs,
        "resolution": resolution
    }
    with open(f"{filename}_metadata.json", "w") as f:
        json.dump(metadata, f)

def download_image(wms, layer, bbox, size, format, filename, crs, resolution):
    response = wms.getmap(
        layers=[layer],
        srs=crs,
        bbox=bbox,
        size=size,
        format=format,
        transparent=True
    )
    img = Image.open(BytesIO(response.read()))
    img.save(filename)
    
    # Speichern der Metadaten
    save_metadata(filename, bbox, size, crs, resolution)

# WMS-Server-URL
url = "https://geodienste.bremen.de/wms_dop10_2023"

# Verbindung zum WMS-Server herstellen
wms = WebMapService(url, version='1.3.0')

# Layer auswählen
layer = 'DOP10_2023_HB'

# Bounding Box definieren (minx, miny, maxx, maxy)
bbox = (488608.7545, 5881989.4935, 490215.9319, 5883625.7616)

# Verzeichnisse für die Ausgabe erstellen
output_dir_tiff = "wms_output_300x300_tiff"
output_dir_png = "wms_output_300x300_png"
os.makedirs(output_dir_tiff, exist_ok=True)
os.makedirs(output_dir_png, exist_ok=True)

# Zielgröße der Kacheln in Pixeln
tile_size = 300

# Zielauflösung in Metern pro Pixel
resolution = 0.1  # Beispielwert, bitte anpassen

# CRS
crs = 'EPSG:25832'

# Breite und Höhe der Bounding Box in Metern
bbox_width = bbox[2] - bbox[0]
bbox_height = bbox[3] - bbox[1]

# Anzahl der Kacheln in x- und y-Richtung
num_tiles_x = int(bbox_width / (tile_size * resolution))
num_tiles_y = int(bbox_height / (tile_size * resolution))

# Breite und Höhe jeder Kachel in Metern
tile_width = bbox_width / num_tiles_x
tile_height = bbox_height / num_tiles_y

for i in range(num_tiles_y):
    for j in range(num_tiles_x):
        tile_bbox = (
            bbox[0] + j * tile_width,
            bbox[1] + i * tile_height,
            bbox[0] + (j + 1) * tile_width,
            bbox[1] + (i + 1) * tile_height
        )
        
        tile_filename = f"tile_{int(tile_bbox[0])}_{int(tile_bbox[1])}"
        download_image(wms, layer, tile_bbox, (tile_size, tile_size), 'image/tiff', f"{output_dir_tiff}/{tile_filename}.tiff", crs, resolution)
        download_image(wms, layer, tile_bbox, (tile_size, tile_size), 'image/png', f"{output_dir_png}/{tile_filename}.png", crs, resolution)

print("Alle Bilder und Metadaten wurden erfolgreich heruntergeladen.")


In [ ]:
import json
import os
from shapely.geometry import Polygon
from pyproj import Transformer
import re
from pyproj import Transformer, CRS

def extract_coordinates(filename):
    # Extrahiert die Koordinaten aus dem Dateinamen
    match = re.search(r'tile_(\d+)_(\d+)', filename)
    if match:
        return f"{match.group(1)}_{match.group(2)}"
    return None

def find_matching_metadata(metadata_dir, coordinates):
    # Sucht nach einer passenden Metadaten-Datei im Verzeichnis
    for filename in os.listdir(metadata_dir):
        if coordinates in filename and filename.endswith('_metadata.json'):
            return os.path.join(metadata_dir, filename)
    return None

def pixel_to_coord(px, py, bbox, img_width, img_height):
    world_width = bbox[2] - bbox[0]
    world_height = bbox[3] - bbox[1]
    x = bbox[0] + (px / img_width) * world_width
    y = bbox[3] - (py / img_height) * world_height
    return x, y

def create_geojson(model_output_file, metadata_dir):
    with open(model_output_file, 'r') as f:
        model_output = json.load(f)

    features = []

    for image in model_output['images']:
        image_file = os.path.basename(image['file_name'])
        coordinates = extract_coordinates(image_file)
        if not coordinates:
            print(f"Konnte keine Koordinaten aus {image_file} extrahieren. Überspringe dieses Bild.")
            continue

        metadata_path = find_matching_metadata(metadata_dir, coordinates)
        if not metadata_path:
            print(f"Keine passenden Metadaten für {image_file} gefunden. Überspringe dieses Bild.")
            continue

        with open(metadata_path, 'r') as f:
            metadata = json.load(f)

        bbox = metadata['bbox']
        crs = metadata['crs']

        # Hier ist die Änderung:
        transformer = Transformer.from_crs(CRS.from_string(crs), CRS.from_epsg(4326), always_xy=True)

        annotations = [ann for ann in model_output['annotations'] if ann['image_id'] == image['id']]

        for ann in annotations:
            if ann['category_id'] == 1:  # Nur Dächer verarbeiten
                world_coords = [pixel_to_coord(x, y, bbox, image['width'], image['height']) 
                                for x, y in zip(ann['segmentation'][0][::2], ann['segmentation'][0][1::2])]
                
                wgs84_coords = [transformer.transform(x, y) for x, y in world_coords]

                feature = {
                    "type": "Feature",
                    "geometry": {
                        "type": "Polygon",
                        "coordinates": [wgs84_coords]
                    },
                    "properties": {
                        "image_id": ann['image_id'],
                        "annotation_id": ann['id'],
                        "area": ann['area']
                    }
                }
                features.append(feature)

    geojson = {
        "type": "FeatureCollection",
        "features": features
    }

    return geojson


# Verwendung der Funktion
model_output_file = r'C:\Users\Marike\Desktop\Hochschule Bremen\2. Semester\2.3 Projekt\Warmelyse\Georeferencing\result.json'
metadata_dir = r'C:\Users\Marike\Desktop\Hochschule Bremen\2. Semester\2.3 Projekt\Warmelyse\NEW\wms_output_300x300_png'
geojson_result = create_geojson(model_output_file, metadata_dir)

# Speichern der GeoJSON-Datei
with open('roofs.geojson', 'w') as f:
    json.dump(geojson_result, f)